## Deliverable 2. Create a Customer Travel Destinations Map.

In [37]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Des,Date
0,0,Sitka,US,57.0531,-135.3300,35.60,75,20,10.36,few clouds,2021-03-28 16:07:31
1,1,Sao Filipe,CV,14.8961,-24.4956,69.91,74,3,15.12,clear sky,2021-03-28 16:09:17
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,89.60,74,20,12.66,few clouds,2021-03-28 16:09:17
3,3,Dingle,PH,10.9995,122.6711,76.86,86,80,5.95,broken clouds,2021-03-28 16:09:18
4,4,Albany,US,42.6001,-73.9662,50.00,85,100,13.73,moderate rain,2021-03-28 16:07:35


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 75


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Des,Date
1,1,Sao Filipe,CV,14.8961,-24.4956,69.91,74,3,15.12,clear sky,2021-03-28 16:09:17
4,4,Albany,US,42.6001,-73.9662,50.00,85,100,13.73,moderate rain,2021-03-28 16:07:35
5,5,Qianan,CN,44.9932,124.0581,42.91,50,7,15.50,clear sky,2021-03-28 16:09:18
6,6,Ribeira Grande,PT,38.5167,-28.7000,62.60,72,40,13.80,scattered clouds,2021-03-28 16:09:18
7,7,Busselton,AU,-33.6500,115.3333,60.01,91,0,8.43,clear sky,2021-03-28 16:09:18
10,10,Lasa,IT,46.6166,10.7002,68.00,26,0,5.75,clear sky,2021-03-28 16:09:19
12,12,Bluff,NZ,-46.6000,168.3333,53.01,97,100,4.50,overcast clouds,2021-03-28 16:09:20
13,13,East London,ZA,-33.0153,27.9116,66.20,68,20,10.36,few clouds,2021-03-28 16:09:20
14,14,Hilo,US,19.7297,-155.0900,66.20,88,90,5.75,moderate rain,2021-03-28 16:09:20
21,21,Kapaa,US,22.0752,-159.3190,73.00,78,90,19.57,overcast clouds,2021-03-28 16:06:27


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        279
City           279
Country        275
Lat            279
Lng            279
Max Temp       279
Humidity       279
Cloudiness     279
Wind Speed     279
Weather Des    279
Date           279
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Des", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Des,Lat,Lng,Hotel Name
1,Sao Filipe,CV,69.91,clear sky,14.8961,-24.4956,
4,Albany,US,50.00,moderate rain,42.6001,-73.9662,
5,Qianan,CN,42.91,clear sky,44.9932,124.0581,
6,Ribeira Grande,PT,62.60,scattered clouds,38.5167,-28.7000,
7,Busselton,AU,60.01,clear sky,-33.6500,115.3333,
10,Lasa,IT,68.00,clear sky,46.6166,10.7002,
12,Bluff,NZ,53.01,overcast clouds,-46.6000,168.3333,
13,East London,ZA,66.20,few clouds,-33.0153,27.9116,
14,Hilo,US,66.20,moderate rain,19.7297,-155.0900,
21,Kapaa,US,73.00,overcast clouds,22.0752,-159.3190,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel no found...skipping")
        

Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping
Hotel no found...skipping


In [35]:

# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna("")
hotel_df.head(10)


,City,Country,Max Temp,Weather Des,Lat,Lng,Hotel Name
1,Sao Filipe,CV,69.91,clear sky,14.8961,-24.4956,Tortuga B&B
4,Albany,US,50.00,moderate rain,42.6001,-73.9662,
5,Qianan,CN,42.91,clear sky,44.9932,124.0581,Qian'an Hotel
6,Ribeira Grande,PT,62.60,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
7,Busselton,AU,60.01,clear sky,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
675,Carovigno,IT,64.40,clear sky,40.7080,17.6550,Hotel Resort Corte di Ferro Puglia Hotels
676,Coolum Beach,AU,68.00,overcast clouds,-26.5333,153.1000,Palmer Coolum Resort
677,Batemans Bay,AU,57.99,overcast clouds,-35.7167,150.1833,Batemans Bay Marina Resort
678,Soller,ES,69.80,clear sky,39.7662,2.7152,Hotel Marina & Wellness Spa


In [56]:
#hotel_df["Hotel Name"].isnull()
#df = pd.DataFrame({"name": ['Alfred', 'Batman', 'Catwoman'],
                   "toy": [np.nan, 'Batmobile', 'Bullwhip'],
                   "born": [pd.NaT, pd.Timestamp("1940-04-25"),
                            pd.NaT]})

,City,Country,Max Temp,Weather Des,Lat,Lng,Hotel Name
4,Albany,US,50.00,moderate rain,42.6001,-73.9662,
116,Zachepylivka,UA,48.15,few clouds,49.1939,35.2394,
288,Ikalamavony,MG,70.02,clear sky,-21.1500,46.5833,
312,Abu Samrah,SY,54.28,clear sky,35.3029,37.1841,
321,Varhaug,NO,46.40,broken clouds,58.6147,5.6457,
353,Vyshkov,RU,55.40,broken clouds,52.4764,31.6873,
376,Awjilah,LY,69.31,clear sky,29.1081,21.2869,
415,Pryamitsyno,RU,44.60,clear sky,51.6561,35.9314,
523,Longonjo,AO,67.08,moderate rain,-12.9067,15.2533,
547,Tongzi,CN,69.96,light rain,28.7829,105.0877,


In [57]:
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Des,Lat,Lng,Hotel Name
1,Sao Filipe,CV,69.91,clear sky,14.8961,-24.4956,Tortuga B&B
5,Qianan,CN,42.91,clear sky,44.9932,124.0581,Qian'an Hotel
6,Ribeira Grande,PT,62.60,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
7,Busselton,AU,60.01,clear sky,-33.6500,115.3333,Observatory Guest House
10,Lasa,IT,68.00,clear sky,46.6166,10.7002,Pension Feldgärtenhof
12,Bluff,NZ,53.01,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
13,East London,ZA,66.20,few clouds,-33.0153,27.9116,Tu Casa
14,Hilo,US,66.20,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
21,Kapaa,US,73.00,overcast clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
22,Azogues,EC,66.20,light rain,-2.7333,-78.8333,Hotel Ceasar Palace


In [59]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/clean_hotels.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [60]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Des}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [63]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [64]:
clean_hotel_df.head()

,City,Country,Max Temp,Weather Des,Lat,Lng,Hotel Name
1,Sao Filipe,CV,69.91,clear sky,14.8961,-24.4956,Tortuga B&B
5,Qianan,CN,42.91,clear sky,44.9932,124.0581,Qian'an Hotel
6,Ribeira Grande,PT,62.60,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
7,Busselton,AU,60.01,clear sky,-33.6500,115.3333,Observatory Guest House
10,Lasa,IT,68.00,clear sky,46.6166,10.7002,Pension Feldgärtenhof
